## Часть 3. Оркестрация пайплайна (Prefect как аналог Airflow)

Построим pipeline:

- check_file — проверка наличия steam_reviews.csv;

- spark_task — Spark-задача, которая фильтрует пользователей раннего доступа, считает среднее время игры (в часах),
сохраняет витрину в формате CSV;

- export_report — вывод результата пользователю.

Все три шага объединяются в один flow (steam_pipeline), который можно запускать по расписанию.

In [1]:
from prefect import flow, task
import os

FILE = "data/steam_reviews.csv"
OUT = "data/steam_early_access_result.csv"


@task
def check_file(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Файл не найден: {path}")
    size = os.path.getsize(path) / (1024 * 1024)
    print(f"[check_file] OK ({size:.2f} MB)")
    return path


@task
def spark_task(path: str):
    """
    Spark-задача: фильтруем только early-access,
    считаем среднее время в часах,
    сохраняем CSV
    """

    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, avg

    spark = (
        SparkSession.builder
        .appName("SteamEarlyAccessPrefect")
        .master("local[*]")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "127.0.0.1")
        .getOrCreate()
    )

    df = spark.read.csv(path, header=True, inferSchema=True)

    df = df.withColumn(
        "written_during_early_access",
        (col("written_during_early_access") == "True")
    )

    df = df.withColumn(
        "playtime",
        col("`author.playtime_forever`").cast("double")
    )

    early = df.filter(col("written_during_early_access") == True)

    result = early.select(
        (col("playtime") / 60).alias("hours")
    ).agg(
        avg("hours").alias("avg_hours")
    )

    result.toPandas().to_csv(OUT, index=False)

    print(f"[spark_task] витрина сохранена в: {OUT}")
    return OUT


@task
def export_report(csv_path: str):
    print(f"[export_report] итоговый CSV: {csv_path}")
    return csv_path


@flow
def steam_pipeline(path: str):
    src = check_file(path)
    mart = spark_task(src)
    export_report(mart)


# запуск flow
steam_pipeline(FILE)


05:50:44.575 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8915
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.

05:50:48.858 | INFO    | Flow run 'chirpy-worm' - Beginning flow run 'chirpy-worm' for flow 'steam-pipeline'

[check_file] OK (7793.22 MB)


05:50:48.879 | INFO    | Task run 'check_file-b86' - Finished in state Completed()

[spark_task] витрина сохранена в: data/steam_early_access_result.csv


05:51:14.230 | INFO    | Task run 'spark_task-131' - Finished in state Completed()

[export_report] итоговый CSV: data/steam_early_access_result.csv


05:51:14.240 | INFO    | Task run 'export_report-018' - Finished in state Completed()

05:51:14.263 | INFO    | Flow run 'chirpy-worm' - Finished in state Completed()